# 🤖 Telegram Bot - Google Colab Runner

Run your **Without-id-pw-appx-cp-uploader** Telegram bot directly from Google Colab.

### ⚡ Instructions:
1. Run each cell **one by one** from top to bottom
2. Cell 1 will **auto clone** from GitHub (or **pull updates** if already cloned)
3. Cell 4 starts the bot — keep this tab open!

> ⚠️ Colab sessions disconnect after ~90 min idle or ~12 hrs max

> 💡 **Workflow:** Edit code locally → `git push` → Re-run Cell 1 (it will `git pull`) → Run Cell 4

## 📦 Cell 1: Clone / Update from GitHub + Install Dependencies

In [ ]:
import os

REPO_URL = 'https://github.com/nat-king-15/Without-id-pw-appx-cp-uploader--main.git'
REPO_DIR = '/content/Without-id-pw-appx-cp-uploader--main'

if os.path.exists(REPO_DIR):
    print('📂 Repo already exists — pulling latest updates...')
    !cd {REPO_DIR} && git pull origin main
    print('✅ Updated to latest version!')
else:
    print('📥 Cloning repo from GitHub...')
    !git clone {REPO_URL} {REPO_DIR}
    print('✅ Repo cloned!')

os.chdir(REPO_DIR)
print(f'\n📍 Working directory: {os.getcwd()}')
print(f'📁 Files: {os.listdir(".")}')

## 📦 Cell 2: Install System + Python Dependencies

In [ ]:
# System deps
!apt-get update -qq && apt-get install -y -qq ffmpeg aria2 > /dev/null 2>&1
print('✅ ffmpeg and aria2 installed!')

# Python deps
!pip install -q pyrogram pyromod tgcrypto aiohttp aiofiles cloudscraper \
    pycryptodome requests m3u8 yt-dlp PyPDF2 reportlab motor umongo \
    speedtest-cli flask gunicorn pytz python-dotenv beautifulsoup4 \
    google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client \
    websockets pytube mutagen bs4
print('✅ All Python packages installed!')

## 🔑 Cell 3: Set Environment Variables

Your config values are pre-filled. Change them if needed.

In [ ]:
import os

# ===== BOT CONFIGURATION =====
os.environ['API_ID']    = '6886135'
os.environ['API_HASH']  = 'ee20a1c8a8e44eaa638b7254cbcc3012'
os.environ['BOT_TOKEN'] = '8525803667:AAEjpylk_qaZ62O_gIZmch46S8VQNYxcY1M'
os.environ['PASS_DB']   = '721'

print('✅ Environment variables set!')
print(f'   API_ID:    {os.environ["API_ID"]}')
print(f'   API_HASH:  {os.environ["API_HASH"][:8]}...')
print(f'   BOT_TOKEN: {os.environ["BOT_TOKEN"][:10]}...')

## 🚀 Cell 4: Run the Bot!

Bot will start and connect to Telegram. **Keep this tab open!**

To stop: click the ⏹️ stop button on this cell.

In [ ]:
import os
REPO_DIR = '/content/Without-id-pw-appx-cp-uploader--main'
os.chdir(REPO_DIR)

# Delete old session file for fresh start
if os.path.exists('bot.session'):
    os.remove('bot.session')
    print('🗑️ Old session file deleted')

print('🚀 Starting bot...')
print('=' * 50)
!python main.py

---
## 🔄 Optional: Keep Colab Alive

Open browser console (F12 → Console) and paste:
```javascript
function KeepAlive() {
    console.log('Keeping alive...');
    document.querySelector('colab-connect-button').click();
}
setInterval(KeepAlive, 60000);
```